In [1]:
import os

In [ ]:
%pwd

In [3]:
os.chdir("../")

In [ ]:
%pwd

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    is_ggdriver: bool
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    dataset_dir: Path
    train_val_test_ratio: list

In [6]:
from src.Classification.constants import *
from src.Classification.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        params = self.params

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            is_ggdriver= config.is_googledriver,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            dataset_dir=config.dataset_dir,
            train_val_test_ratio=params.TRAIN_VAL_TEST_RATIO
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
import shutil
import numpy as np
import matplotlib.pyplot as plt
from src.Classification import logger
from sklearn.model_selection import train_test_split


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.classes = None
    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            if self.config.is_ggdriver:
                file_id = dataset_url.split("/")[-2]
                prefix = 'https://drive.google.com/uc?/export=download&id='
                gdown.download(prefix+file_id, zip_download_dir)
            else:
                shutil.copy(dataset_url, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
            
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        # Remove file .zip
        try:
            os.remove(self.config.local_data_file)
            logger.info(f"File {self.config.local_data_file} has been deleted successfully.")
        except Exception as e:
            logger.error(f"Error occurred while deleting {self.config.local_data_file}: {e}")


    def split_dataset(self):
        """
        Splip data to Train/Validation/Test
        """
        try:
            os.makedirs(self.config.dataset_dir, exist_ok=True)
            data_dir = os.path.join(self.config.unzip_dir, "DefectHole")
            classes = os.listdir(data_dir)
            logger.info(f"Catalog: {classes}")

            ## Show Dataset Count
            counts = []
            for i in range(len(classes)):
                class_dir = data_dir + '/' + classes[i]
                count = len([name for name in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, name))])
                counts.append(count)
            logger.info(f"Count: {counts}")
            x = np.array(classes)
            y = np.array(counts)
            plt.barh(x,y)
            plt.title("Count Datasets by Class")
            plt.savefig(os.path.join(self.config.root_dir, "CountDatasets.png"))

            for split in ['train', 'val', 'test']:
                split_dir = os.path.join(self.config.dataset_dir, split)
                os.makedirs(split_dir, exist_ok=True)
                for class_name in classes:
                    os.makedirs(os.path.join(split_dir, class_name), exist_ok=True)
            
            train_ratio = self.config.train_val_test_ratio[0]
            val_ratio = self.config.train_val_test_ratio[1]
            test_ratio = self.config.train_val_test_ratio[2]

            if (train_ratio + val_ratio + test_ratio != 1):
                logger.error("Ratio Value Error! Expected sum of ratios = 1")
                raise Exception("Ratio Value Error! Expected sum of ratios = 1")
            
            logger.info("Splitting dataset -> Train/Validation/Test")
            dataset_size = 0

            for class_name in classes:
                source_dir = os.path.join(data_dir, class_name)
                images = os.listdir(source_dir)
                dataset_size = dataset_size + len(images)

                train, temp = train_test_split(images, test_size=(1 - train_ratio), random_state=42)
                val, test = train_test_split(temp, test_size=(1 - val_ratio / (val_ratio + test_ratio)), random_state=42)
                
                for split, split_images in zip(['train', 'val', 'test'], [train, val, test]):
                    for image in split_images:
                        src_path = os.path.join(source_dir, image)
                        dest_path = os.path.join(self.config.dataset_dir, split, class_name, image)
                        shutil.copy(src_path, dest_path)

            logger.info(f"Size of dataset: {dataset_size} images")
        except Exception as e:
            raise e

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.split_dataset()
except Exception as e:
    raise